In [30]:
# Import Libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
# Import csv data
def load_dataset(fileName):
    allData = pd.read_csv('./' + fileName)
    return allData

In [3]:
# Get image file names from folders
def get_img_files(img_path):
    img_files = []
    for file in os.listdir(img_path):
        if file.endswith('.jpg'):
            img_files.append(file.replace('.jpg','')) # Strip .jpg extension from filenames - return img ID
    return img_files

In [4]:
# Preprocess all image files
def preprocess_img(img, img_path, img_size):
    image = load_img(os.path.join(img_path, img + '.jpg'), grayscale=False, color_mode='rgb', target_size=img_size)
    img_array = img_to_array(image)

    # normalize img_array
    img_array = img_array.astype('float32')
    img_array = (img_array / 255.0 - 0.5) * 2.0 # normalize by 255, then center and scale up to be from -1 to 1

    return img_array

In [53]:
def define_img_model(num_outputs, input_shape):
    input_tensor = tf.keras.layers.Input(shape=input_shape)
    model = tf.keras.applications.mobilenet.MobileNet(
        input_tensor = input_tensor,
        alpha = 1.0,
        weights = None,
        include_top = False
    )
    output_reshape = tf.keras.layers.Reshape((4096,))(model.output)
    output_dense = tf.keras.layers.Dense(200, activation='relu')(output_reshape)
    output_layer = tf.keras.layers.Dense(num_outputs, activation='softmax')(output_dense)

    final_model = tf.keras.Model(inputs=model.input, outputs=output_layer)
    return final_model



In [45]:
# model evaluation
def evaluate_model(model, trainX, trainY, testX, testY, num_epochs):
    verbose = 1
    scores, histories = list(), list()

    history = model.fit(trainX, trainY, epochs=num_epochs, validation_data=(testX,testY), verbose = verbose)
    # evaluate model performance
    _, acc = model.evaluate(testX, testY, verbose=1)
    print ('> %.3f' % (acc * 100.0))

    scores.append(acc)
    histories.append(history)

    return scores, histories

In [46]:
# present results
def summarize_diagnostics(histories):
    for i in range(len(histories)):
        #plot loss 
        plt.subplot(2,1,1)
        plt.title('Cross Entropy Loss')
        plt.plot(histories[i].history['loss'], color='blue', label='train')
        plt.plot(histories[i].history['val_loss'], color='orange', label='test')
        plt.legend()
        # plot accuracy
        plt.subplot(2,1,2)
        plt.title('Classification Accuracy')
        plt.plot(histories[i].history['accuracy'], color='blue', label='train')
        plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
        plt.legend()
    plt.show()

In [27]:
# Load in csv data
txtData = load_dataset('./train.csv')
train_img_path = './train/'
test_img_path = './test/'

train_img_files = get_img_files(train_img_path)
test_img_files = get_img_files(test_img_path)

# train_img_files = train_img_files[:2000] # for testing

# Isolate out unique categories
allCategories = pd.DataFrame(txtData['categories']).drop_duplicates(ignore_index=True)

In [28]:
# Read in image files
train_images = []
ylabels = []
img_size = (80,80)

# loop and read all image files into array
for img in train_img_files:
    img_array = preprocess_img(img, train_img_path, img_size)
    
    if img_array is not None:
        train_images.append(img_array.astype('float32'))

        # pull ylabel category
        img_index = txtData[txtData['ImgId'] == img].index.values[0]
        txt_row = txtData.iloc[img_index,:]
        category = txt_row[3]
        # description = txt_row[2]
        # title = txt_row[1]
        # ylabels.append(list(allCategories['categories']).index(category))
        ylabels.append(category)
        # print(title + '-' + category)

train_images = np.asarray(train_images)
ylabels = pd.get_dummies(ylabels).values


In [42]:
# Split data into training and testing data
img_trainX, img_testX, img_trainY, img_testY = train_test_split(train_images, ylabels, test_size = 0.3, random_state=0)

(29400, 80, 80, 3)


In [54]:
# Define model
input_shape = img_size + (3,)
model = define_img_model(num_outputs=ylabels.shape[1], input_shape=input_shape)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 80, 80, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 40, 40, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 40, 40, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 40, 40, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 40, 40, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 40, 40, 32)       128       
 ation)                                                      

In [55]:
# Train model
num_epochs = 10
scores, histories = evaluate_model(model, img_trainX, img_trainY, img_testX, img_testY, num_epochs)

Epoch 1/10
919/919 [==============================] - 750s 809ms/step - loss: 3.0404 - accuracy: 0.0555 - val_loss: 3.0001 - val_accuracy: 0.0607
Epoch 2/10
919/919 [==============================] - 478s 520ms/step - loss: 2.9722 - accuracy: 0.0784 - val_loss: 3.0588 - val_accuracy: 0.0687
Epoch 3/10
919/919 [==============================] - 475s 517ms/step - loss: 2.9369 - accuracy: 0.0842 - val_loss: 2.9577 - val_accuracy: 0.0773
Epoch 4/10
919/919 [==============================] - 475s 516ms/step - loss: 2.9164 - accuracy: 0.0867 - val_loss: 2.9269 - val_accuracy: 0.0784
Epoch 5/10
919/919 [==============================] - 604s 657ms/step - loss: 2.9028 - accuracy: 0.0894 - val_loss: 2.9427 - val_accuracy: 0.0792
Epoch 6/10
349/919 [==========>...................] - ETA: 8:00 - loss: 2.8819 - accuracy: 0.1032

KeyboardInterrupt: 

In [ ]:
# Diagnostics
summarize_diagnostics(histories)

NameError: name 'histories' is not defined